# Question 1

Write a function to get the last item in a complete tree. This is easy to do if the complete tree were implemented using arrays. How would we do this if the tree was implemented using nodes?

In [1]:
def get_last_array(tree:list):
    if type(tree) == list:
        return tree[-1]
    else:
        raise TypeError("Input type must be a list")   

In [2]:
from dsa.tree import Node, Tree
from dsa.queue import Queue

root = Node(1)
root.left = Node(2)
root.right = Node(3)
root.left.left = Node(4)
root.left.right = Node(5)
root.right.left = Node(6)
root.right.right = Node(7)
root.left.left.left = Node(8)
root.left.left.right = Node(9)
root.left.right.left = Node(10)
root.left.right.right = Node(11)
root.right.left.left = Node(12)
root.right.left.right = Node(13)
root.right.right.left = Node(14)
root.right.right.right = Node(15)

tree = Tree(root)


tree.print()

         15
      7
         14
   3
         13
      6
         12
1
         11
      5
         10
   2
         9
      4
         8


In [13]:
q =Queue()
q.enqueue(tree.root)

a = q.dequeue()

a.value


1

In [26]:
def find_last_node(tree:Tree):
    
    root = tree.root
    
    if root.value == None:
        return None
    
    q = Queue()
    q.enqueue(root)

    last_node = q.peek()

    while q.is_empty() == False:
        last_node = q.dequeue()
        if last_node.left:
            q.enqueue(last_node.left)
        if last_node.right:
            q.enqueue(last_node.right)
        
    return last_node.value

find_last_node(tree)
        

15

The above functions return the final item in a complete binary tree. The array version simply returns the final index of an array, which uses array indexing so uses $O(1)$ time and $O(1)$ space, since no new data is being created or stored.

The tree version works by implementing level order traversal. We cannot simply traverse down the left/right to find the last level and then search it because in a tree, values are referenced using nodes, meaning there's no way to go left or right in a tree, you can only go up/down. Thus, we cleverly leverage queues to enqueue and dequeue children and update our last seen node. We enqueue the root when initializing the algorithm. 

We then while loop through the tree, deuqueing the current node. The current node's left and right children are then added to the back of the queue. The next item in the dequeued will be in the same level as the parent of the child nodes just enqueued, because the tree is complete, and this process continues until a level has been exhausted. Since we are traversing each level from left to right and the tree is guaranteed to be complete, the last node visited will be the final value, and the last node is updated to be the current node everytime a new node is dequeued. This algorithm has a time complexity of $O(n)$ since we are visiting every node once. It uses at most $\frac{n}{2}$ nodes in the queue, which means that overall the algorithm uses $O(n)$ space.

# Question 2